# Get Images from video

In [1]:
import cv2
import os

# Input Path
video_path = 'cafe.mp4'

# Output Path
output_dir = 'training/person_1'

# Create the output directory if it doesn't exist
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Create a VideoCapture object to read the input video
cap = cv2.VideoCapture(video_path)

# Get the total number of frames in the video
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

# Calculate the frame interval to capture for 150 images
frame_interval = total_frames // 1000 # change this number according to your needs 

# Set the initial frame counter to 0
frame_counter = 0

while cap.isOpened():
    # Read a frame from the video
    ret, frame = cap.read()
    
    if not ret:
        break
    
    # Check if this is the frame to capture
    if frame_counter % frame_interval == 0 and frame_counter // frame_interval < 1000:
        # Save the frame as a JPEG image
        output_path = os.path.join(output_dir, f'{frame_counter//frame_interval + 1:03}.jpg')
        cv2.imwrite(output_path, frame)
    
    # Increment the frame counter
    frame_counter += 1
    
    if frame_counter >= total_frames:
        break

# Release the video capture object
cap.release()
print("\n\nDONE\n\n")



DONE




# Get image from Internet

In [ ]:
from simple_image_download import simple_image_download as simp

# from simp library call simple_image_download function
response = simp.simple_image_download

# the keywords that will be used to find pics, and each key work will create a different file 
keywords = ["George Wassouf", "Donald Trump", "Selena Gomez"]

# for loop on the keywords
# (kw, 1000) means 300 sample of each keyword 
for kw in keywords:
    response().download(kw, 1000) 

In [ ]:
# Face Detection with MTCNN (Multi-Task Cascaded Convolutional Neural Network)
from os import listdir
from os.path import isdir
from PIL import Image
from matplotlib import pyplot
from numpy import savez_compressed
from numpy import asarray
from mtcnn.mtcnn import MTCNN

# extract a single face from a given photograph
def extract_face(filename, required_size=(160, 160)):
    # load image from file
    image = Image.open(filename)
    # convert to RGB, if needed
    image = image.convert('RGB')
    # convert to array
    pixels = asarray(image)
    # create the detector, using default weights
    detector = MTCNN()
    # detect faces in the image
    results = detector.detect_faces(pixels)
    # check if face was detected
    if results:
        # extract the bounding box from the first face
        x1, y1, width, height = results[0]['box']
        # bug fix
        x1, y1 = abs(x1), abs(y1)
        x2, y2 = x1 + width, y1 + height
        # extract the face
        face = pixels[y1:y2, x1:x2]
        # resize pixels to the model size
        image = Image.fromarray(face)
        image = image.resize(required_size)
        face_array = asarray(image)
        return face_array
    else:
        return None

# load images and extract faces for all images in a directory
def load_faces(directory):
    faces = []
    # enumerate files
    for filename in listdir(directory):
        # path
        path = directory + filename
        # get face
        face = extract_face(path)
        # check if face was detected
        if face is not None:
            # store
            faces.append(face)
    return faces

# load a dataset that contains one subdir for each class that in turn contains images
def load_dataset(directory):
    X, y = [], []
    # enumerate folders, on per class
    for subdir in listdir(directory):
        # path
        path = directory + subdir + '/'
        # skip any files that might be in the dir
        if not isdir(path):
            continue
        # load all faces in the subdirectory
        faces = load_faces(path)
        # create labels
        labels = [subdir for _ in range(len(faces))]
        # summarize progress
        print('>loaded %d examples for class: %s' % (len(faces), subdir))
        # store
        X.extend(faces)
        y.extend(labels)
    return asarray(X), asarray(y)

# load train dataset
trainX, trainy = load_dataset('/home/jawabreh/Desktop/face-recognition/training/')
print(trainX.shape, trainy.shape)
# save arrays to one file in compressed format
savez_compressed('/home/jawabreh/Desktop/face-recognition/face-detection.npz', trainX, trainy)